In [14]:
import pandas as pd
import numpy as np
import datetime as datetime

In [15]:
df = pd.read_csv(r'input.csv', index_col = False, header = None)
df.columns = ['TimeStamp','Symbol','Quantity','Price']

In [16]:
#Solve MaxTimeGap 
MaxTimeGap_df = df
MaxTimeGap_df['MaxTimeGap'] = df.groupby(by = ['Symbol'])['TimeStamp'].diff().round()
MaxTimeGap_df = pd.DataFrame(MaxTimeGap_df.groupby(by = ['Symbol'])['MaxTimeGap'].max()).astype(int)

In [17]:
#Solve Weighted Average Price
def vwap(df):
    q = df.Quantity.values
    p = df.Price.values
    return df.assign(vwap=(p * q).cumsum() / q.cumsum())

WeightedAveragePrice_Full = df.groupby(by = 'Symbol', group_keys=True).apply(vwap) #rolling vwap stored for reference
WeightedAveragePrice_df = WeightedAveragePrice_Full.drop_duplicates(subset=["Symbol"], keep="last").drop(['Symbol'],axis=1) #only concerned with full interval vwap
WeightedAveragePrice_df = WeightedAveragePrice_df.reset_index()[['Symbol','vwap']].set_index(['Symbol']).rename(columns={'vwap':'WeightedAveragePrice'}).astype(int)

In [18]:
#Max price and total volume
grouped = df.groupby('Symbol')
Volume = pd.DataFrame(grouped.sum().Quantity).rename(columns={'Quantity':'Volume'})
MaxPrice = pd.DataFrame(grouped.max().Price).rename(columns={'Price':'MaxPrice'})

combined_df = Volume.merge(MaxPrice, on = 'Symbol')

In [19]:
#merge final results and save
result_df = (WeightedAveragePrice_df.merge(combined_df, on = ['Symbol']).merge(MaxTimeGap_df, on = ['Symbol'])).reset_index()
result_df = result_df[['Symbol','MaxTimeGap','Volume','WeightedAveragePrice','MaxPrice']]
result_df.to_csv(r'output.csv', index = False, header=False)